# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2025</br>

Name: Joe D'Agostino
</br>
Date: 4/10/25
<br>
<br>
**At this time in the semester:** <br>
- We have explored a dataset. <br>
- We have cleaned our dataset. <br>
- We created a Github account with a repository for this class and included a metadata read me file about our data. <br>
- We introduced general SQL syntax, queries, and applications in Python.<br>
- Created our own databases from scratch using MySQL Workbench and Python with SQLAlchemy on our local server and locally on our machine.
- Populated our databases with the data we cleaned at the start of the semester.
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source, especially when posting your assignments to Github.<br>

Feel free to use your preferred library and method for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook!<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. Please submit this assignment with a link to it posted to your Github.<br>

**Data origin:** I'm using the usa_olympic_athletes_7_2 MySQL database I created in Assignment 7 using my [USA Olympic Athletes](https://github.com/joedag32/DSSA-5102_Spring2025/blob/main/README.md) dataset.

### All below queries ran through MySQL Workbench

#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** What age group are athletes in? 20s, 30s, etc.?
<br>
**What tables are we joining? (If any):** athlete

In [ ]:
SELECT name,
	CASE
		WHEN age > 40 THEN 'Over 40'
        WHEN age >= 30 THEN 'In their 30\'s'
        WHEN age >= 20 THEN 'In their 20\'s'
		ELSE 'Teenager'
	END AS age_group
FROM athlete
GROUP BY name, age_group;


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** What endorsement opportunities does an athlete probably get based on what medal they earned
<br>
**What tables are we joining? (If any):** athlete and medal

In [ ]:
SELECT name, year,
	CASE
		WHEN medal.medal_name = 'Gold' THEN 'Wheaties box'
        WHEN medal.medal_name = 'Silver' THEN 'TV commercial'
		ELSE 'Local library appearance'
	END AS endorsement
FROM medal
INNER JOIN athlete ON medal.medal_id = athlete.medal_id
WHERE medal.medal_name != '0'
ORDER BY year DESC;


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** 
<br>
**What tables are we joining? (If any):** 

**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Question:** Get all the rows where the weight or height value is NULL
<br>
**What tables are we joining? (If any):** athlete

In [ ]:
SELECT *
FROM athlete 
WHERE weight IS NULL 
OR height IS NULL;


I don't have any NULL values in either of those columns due to clean up, but that would return rows if they were there.

**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Insert Olympic entries for me and Cliff into the athlete table (I'm completely guessing on 
<br>
**What table(s) are we adding a record to?** athlete

In [1]:
-- Joe did not medal in Tennis and Stacey got the gold in featherweight weightlifting
INSERT INTO athlete(name, sex, age, height, weight, team, year, season, city, sport_id, event_id, medal_id) 
VALUES 
  ('Joe', 'M', 47.0, 177.8, 83.9, 'United States', 2024, 'Summer', 'Paris', 26, 65, 1), 
  ('Stacey', 'F', 39.0, 147.3, 47.6, 'United States', 2024, 'Summer', 'Paris', 15, 617, 4);


SyntaxError: invalid syntax (1157326148.py, line 1)

**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above.<br>
The IN operator in a WHERE clause lets you filter for multiple values at once. You can also exclude certain values by using the NOT IN operator.
<br>

**Question:** What were the heaviest and lightest athlete weights grouped by sport?
<br>
**What table(s) are we joining? (If any)** sport and athlete

In [ ]:
SELECT MAX(weight) AS heaviest, MIN(weight) AS lightest, sport_name
FROM sport
INNER JOIN athlete ON sport.sport_id = athlete.sport_id
WHERE weight > 0 -- check to make sure a weight was recorded
GROUP BY sport_name;


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What is tallest male and female US Olympians by year.
<br>
**What table(s) are we joining?** athlete

In [ ]:
SELECT year,
	MAX(CASE WHEN sex = 'F' THEN height ELSE NULL END) as max_female_height,
    MAX(CASE WHEN sex = 'M' THEN height ELSE NULL END) as max_male_height
FROM athlete
GROUP BY year
ORDER BY year DESC;


**MIN() and MAX()** <br>
AVG() will take the average of a numeric field.

**Question:** What's the average female and male athlete age from 1900 to 2000?
<br>
**What table(s) are we joining?** athlete

In [ ]:
SELECT year,
	AVG(CASE WHEN sex = 'F' THEN age ELSE NULL END) as avg_female_age,
    AVG(CASE WHEN sex = 'M' THEN age ELSE NULL END) as avg_male_age
FROM athlete
WHERE year BETWEEN 1900 AND 2000
GROUP BY year
ORDER BY year DESC;


Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above. <br><br>
**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above (If you have not done so already). Be sure to obtain the same result set.

In [ ]:
-- using abbrevations a for the athlete table and m for the medal table
SELECT a.name, a.year,
	CASE
		WHEN m.medal_name = 'Gold' THEN 'Wheaties box'
        WHEN m.medal_name = 'Silver' THEN 'TV commercial'
		ELSE 'Local library appearance'
	END AS endorsement
FROM medal as m
INNER JOIN athlete AS a ON m.medal_id = a.medal_id
WHERE m.medal_name != '0'
ORDER BY year DESC;


I seem to not typically use abbrevations for tables, as I really hope I gave the tables clean names, but I'm sure working with larger datasets or databases I didn't create it would sure make things easier!

Now we are starting to create multiple new fields that we can save any time as a .csv if needed to access later. Save your result set as a .csv:

In [22]:
# Save result set as .csv file:



**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Objective:** Delete the records created for Joe and Stacey from the athlete table from this assignment.
<br>
**What table(s) are we deleting records from?** athlete

In [24]:
-- Our Olympic careers were short but most enjoyable
DELETE FROM athlete
WHERE athlete_id IN (37209, 37210);


We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. It is preferred that you submit your Github link instead of the file itself.